In [5]:
import cv2
import ffmpegcv
from tqdm.auto import tqdm
from ultralytics import YOLO

In [2]:
VIDEO_PATH = '/home/skorp321/Projects/panorama/data/Swiss_vs_slovakia-Panorama.mp4'
MODEL_PATH = '/home/skorp321/Projects/panorama/models/yolov8m_keypoints.pt'

In [ ]:
model = YOLO(MODEL_PATH)

In [ ]:
cap = ffmpegcv.VideoCaptureNV(VIDEO_PATH)

for frame in tqdm(cap):
    outputs = model(frame, save=True)